In [ ]:
import numpy as np
import torch
from traversability_estimation.utils import show_cloud
from traversability_estimation.datasets import TraversabilityDataset
from traversability_estimation.segmentation import filter_grid, filter_range
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

### Load traversability data

In [ ]:
# path = '/home/ruslan/data/bags/traversability/marv/ugv_2022-08-12-15-18-34_trav/'
# path = '/home/ruslan/data/bags/traversability/marv/ugv_2022-08-12-16-37-03_trav/'
path = '/home/ruslan/data/bags/traversability/marv/ugv_2022-08-12-15-30-22_trav/'
assert os.path.exists(path)

ds = TraversabilityDataset(path, cloud_topic='os_cloud_node/destaggered_points')

# visualize a sample from the data set
# for i in np.random.choice(range(len(ds)), 1):
#     _ = ds.__getitem__(i, visualize=True)

print('Dataset contains %i samples' % len(ds))

### Choose data sample

In [ ]:
i = np.random.choice(range(len(ds)))
sample = ds[i]
depth, label, points = sample
traj = ds.get_traj(i)

print('%i-th sample contains range image of shape: %s, its label of shape: %s, point cloud of shape: %s, and traversed trajectory of shape: %s' %
     (i, depth.shape, label.shape, points.shape, traj.shape))

### Filter point cloud and its label

In [ ]:
label_filtered = label.squeeze().reshape((-1,))
points_filtered = points.copy()

min_dist, max_dist = 0.1, 10.0
_, mask = filter_range(points, min_dist, max_dist, return_mask=True)
points_filtered = points_filtered[mask]
label_filtered = label_filtered[mask]

grid_res = 0.2
_, mask = filter_grid(points_filtered, grid_res, return_mask=True)
points_filtered = points_filtered[mask]
label_filtered = label_filtered[mask]

# remove points which are above the robot
h_max = 0.5
mask = points_filtered[:, 2] <= h_max
points_filtered = points_filtered[mask]
label_filtered = label_filtered[mask]

### Create height map and add labels to traversed cells

In [ ]:
H = W = round(2 * max_dist / grid_res)

height_map = np.full((H, W), 0.0)
trav_map = np.full((H, W), np.nan)

height_map.shape

In [ ]:
ids = (points_filtered[:, :2] - points_filtered[:, :2].min(axis=0)) // grid_res
ids = np.asarray(ids, dtype=int)
ids.shape

In [ ]:
min_height = points_filtered[:, 2].min()
for i, (idx, idy) in enumerate(ids):
    height_map[idy, idx] = points_filtered[i, 2] - min_height
    
    # traversability map contains labels only for traversed points
    if label_filtered[i] != 255:
        trav_map[idy, idx] = label_filtered[i]

# robot trajectory transformed to grid
traj_grid = (traj[:, :2, 3] - points_filtered[:, :2].min(axis=0)) / grid_res

### Visualize height map and traversabilty labels

In [ ]:
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.imshow(height_map)
plt.plot(traj_grid[:, 0], traj_grid[:, 1], 'ro')
plt.grid()

plt.subplot(1, 2, 2)
plt.imshow(trav_map)
plt.plot(traj_grid[:, 0], traj_grid[:, 1], 'ro')
plt.grid()

### Visualize height map as a surface and point cloud with robot trajectory

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

x = y = np.arange(-max_dist, max_dist, grid_res)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(121, projection='3d')
ax.plot_surface(X, Y, height_map)
ax.set_zlim3d([-0.1, 10.0])

ax = fig.add_subplot(122, projection='3d')
ax.plot(traj[:, 0, 3], traj[:, 1, 3], traj[:, 2, 3], 'ro', markersize=4)
ax.plot(points_filtered[:, 0], points_filtered[:, 1], points_filtered[:, 2], '.', markersize=1)

plt.show()